In [1]:

# Step 1: Install dependencies
!pip install langchain langchain-groq langgraph requests langchain-experimental==0.3.5rc1


In [3]:
# Step 2: Import libraries
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import AgentType
from langchain_community.tools.requests.tool import RequestsGetTool # Corrected import
import langgraph as lg
import os

In [ ]:
# Step 3: Set Groq API Key
os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"  # 👉 Replace with your actual Groq API key

In [5]:
# Step 4: Initialize LLM (Groq / Gemini)
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.1-8b-instant"
)


In [8]:
# ============================================================
# Step 5: Define Tools for Agent
# ============================================================

# 5.1 Production Efficiency Calculator
def calculate_efficiency(produced_units, target_units):
    return round((produced_units / target_units) * 100, 2)

calculator_tool = Tool(
    name="ProductionEfficiencyCalculator",
    func=lambda x: str(calculate_efficiency(*map(float, x.split(',')))),
    description="Calculates production efficiency. Input: 'produced_units,target_units'"
)

# 5.2 Python REPL Tool for multi-step calculations
python_tool = PythonREPLTool()

# 5.3 HTTP Requests Tool to fetch external data
from langchain_community.utilities import RequestsWrapper # Import RequestsWrapper

requests_wrapper = RequestsWrapper() # Create an instance of RequestsWrapper

http_tool = RequestsGetTool(
    name="InventoryAPI",
    description="Use this tool to fetch inventory or reorder information from external APIs. Input: URL of the API endpoint.",
    requests_wrapper=requests_wrapper, # Pass the requests_wrapper instance
    allow_dangerous_requests=True
)

In [9]:
# ============================================================
# Step 6: Initialize Agent with Tools (ReAct-style)
# ============================================================

tools = [calculator_tool, python_tool, http_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


/tmp/ipython-input-3546944005.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [10]:
# ============================================================
# Step 7: Test Agent Interactions
# ============================================================

# Example Query 1: Production Efficiency
query1 = "If Line A produces 450 units instead of 500, what is the efficiency?"
response1 = agent.run(query1)
print("Query:", query1)
print("Response:", response1)

# Example Query 2: Downtime Cost
query2 = "If downtime per machine is 2 hours and cost per hour is 1500, calculate total downtime cost for 3 machines."
response2 = agent.run(query2)
print("Query:", query2)
print("Response:", response2)

# Example Query 3: Fetch Inventory via API (dummy example)
query3 = "Check current stock for item ID 123 from http://dummy-inventory-api.com/item/123"
response3 = agent.run(query3)
print("Query:", query3)
print("Response:", response3)

/tmp/ipython-input-1416815271.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run(query1)




> Entering new AgentExecutor chain...
{
  "action": "ProductionEfficiencyCalculator",
  "action_input": "450,500"
}

> Finished chain.
Query: If Line A produces 450 units instead of 500, what is the efficiency?
Response: {
  "action": "ProductionEfficiencyCalculator",
  "action_input": "450,500"
}


> Entering new AgentExecutor chain...
{
  "action": "ProductionEfficiencyCalculator",
  "action_input": "10,40"
} 



> Finished chain.
Query: If downtime per machine is 2 hours and cost per hour is 1500, calculate total downtime cost for 3 machines.
Response: {
  "action": "ProductionEfficiencyCalculator",
  "action_input": "10,40"
} 




> Entering new AgentExecutor chain...
Action: {
  "action": "InventoryAPI",
  "action_input": "http://dummy-inventory-api.com/item/123"
}



> Finished chain.
Query: Check current stock for item ID 123 from http://dummy-inventory-api.com/item/123
Response: Action: {
  "action": "InventoryAPI",
  "action_input": "http://dummy-inventory-api.com/item/123"
